In [21]:
import numpy as np
from numpy import asarray
from numpy import arange
from numpy.random import rand
from numpy.random import seed
import matplotlib.pyplot as plt
from func_autograd import *
from sklearn.model_selection import train_test_split

def MSE(y_data, y_model):
	n = np.size(y_model)
	y_data = y_data.reshape(-1,1)
	y_model = y_model.reshape(-1,1)
	return np.sum((y_data - y_model)**2)/n


def generate_data(noise=True, step_size=0.05 , FrankesFunction=True):
    # Arrange x and y
    x = np.arange(0, 1, step_size)
    y = np.arange(0, 1, step_size)
    # Create meshgrid of x and y
    X, Y = np.meshgrid(x, y)
    
    if FrankesFunction:
        # Calculate the values for Franke function
        z = FrankeFunction(X, Y, noise=noise).flatten()
    else:
        z = TestFunction(X, Y, noise=noise).flatten()

    # Flatten x and y for plotting
    x = X.flatten()
    y = Y.flatten()
    
    return x, y, z

def TestFunction(x, y, noise=False):
    if noise: 
        random_noise = np.random.normal(0, 0.1 , x.shape)
    else: 
        random_noise = 0

    return  x**2 + y**2 + 2*x*y + random_noise

def FrankeFunction(x, y, noise=False):
    if noise: 
        random_noise = np.random.normal(0, 0.1 , x.shape)
    else: 
        random_noise = 0
    
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 + random_noise

x, y, z = generate_data()
X = np.column_stack((x,y))
X_train, X_test, z_train, z_test = train_test_split(X, z)

# plain gradient descent with fixed learning rate using analytic expression of gradient
gd = GradientDescend(momentum=False)
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
gd_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
gd_MSE_test = MSE(pred_test, z_test)

# adding momentum
gd = GradientDescend()
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
gd_mom_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
gd_mom_MSE_test = MSE(pred_test, z_test)

# minibatch sgd, learning schedule
# check method
gd = GradientDescend(optimizer="sgd", method="gd")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
sgd_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
sgd_MSE_test = MSE(pred_test, z_test)

# adagrad 
# gd without momentum
gd = GradientDescend(momentum=False, method="adagrad")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
gd_adagrad_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
gd_adagrad_MSE_test = MSE(pred_test, z_test)

# gd with momentum
gd = GradientDescend(method="adagrad")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
gd_mom_adagrad_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
gd_mom_adagrad_MSE_test = MSE(pred_test, z_test)

# sgd without momentum
gd = GradientDescend(optimizer="sgd", momentum=False, method="adagrad")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
sgd_adagrad_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
sgd_adagrad_MSE_test = MSE(pred_test, z_test)

# sgd with momentum
gd = GradientDescend(optimizer="sgd", method="adagrad")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
sgd_mom_adagrad_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
sgd_mom_adagrad_MSE_test = MSE(pred_test, z_test)

# rms prop and adam
# We chose to only evaluate sgd with momentum for this
gd = GradientDescend(optimizer="sgd", method="rmsprop")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
sgd_mom_rmsprop_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
sgd_mom_rmsprop_MSE_test = MSE(pred_test, z_test)

gd = GradientDescend(optimizer="sgd", method="adam")
scores = gd.fit(X_train, z_train, X_test, z_test)

pred_train = gd.predict(X_train)
sgd_mom_adam_MSE_train = MSE(pred_train, z_train)
pred_test = gd.predict(X_test)
sgd_mom_adam_MSE_test = MSE(pred_test, z_test)
